<font color='cadetblue'><font size="4">**IMPORT LIBRARIES AND RATIOS_COMPUTATION FILE**</font>

In [1]:
#FINANCIAL LIBRARIES
import FundamentalAnalysis as fa

#OTHER LIBRARIES
import import_ipynb
import time
from datetime import datetime
import pickle
from itertools import chain
import pandas as pd
import numpy as np
import RATIOS_COMPUTATION as rc

importing Jupyter notebook from RATIOS_COMPUTATION.ipynb


<font color='cadetblue'><font size="4">**PART 1: CREATE INSTANCES OF COMPANY CLASS FOR ALL S&P 500 TICKERS AND SAVE THEM**</font>

In [2]:
class Company:
    
    def __init__(self, ticker, ttm=False, disp=0, api_key="Here would go mi personal API key"):
        
        self.ticker = ticker
        
        self.ttm = ttm
        
        self.disp = disp
        
        #DEFINE FINANCIAL TABLES
        
        self.__profile = fa.profile(ticker, api_key)
        #self.__ratings = fa.rating(ticker, api_key) #NOT NECESSARY BY NOW
        self.__enterprise_value = fa.enterprise(ticker, api_key, period="quarter")
        try:
            self.__dcf = fa.discounted_cash_flow(ticker, api_key, period="quarter")
        except (KeyError, ValueError, IndexError):
            self.__dcf = None
        
            
        self.__balance_sheet = fa.balance_sheet_statement(ticker, api_key, period="quarter")
        self.__income_statement = fa.income_statement(ticker, api_key, period="quarter")
        self.__cash_flow_statement = fa.cash_flow_statement(ticker, api_key, period="quarter")
        self.__key_metrics = fa.key_metrics(ticker, api_key, period="quarter")
        self.__financial_ratios = fa.financial_ratios(ticker, api_key, period="quarter")
        self.__growth = fa.financial_statement_growth(ticker, api_key, period="quarter")
        #self.__stock_data = fa.stock_data(ticker, period="ytd", interval="1d")  #NOT NECESSARY BY NOW
        #self.__stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="1985-01-01", end="2022-01-01")  #NOT NECESSARY BY NOW
        try:
            self.__dividends = fa.stock_dividend(ticker, api_key, begin="1900-01-01", end=datetime.today().strftime('%Y-%m-%d'))
        except (KeyError, ValueError):
            self.__dividends = None
        
        
        #Compute ratios
        self.__compute_ratios__(api_key)
        
    def __compute_ratios__(self, api_key):
        
        #PROFITABILITY ratios
        self.PAT_margin = rc.exception_handler(rc.compute_net_profit_margin, self.__financial_ratios, self.ttm, self.disp)
        self.EBITDA_margin = rc.exception_handler(rc.compute_EBITDA_margin, self.__income_statement, self.ttm, self.disp)
        self.return_on_equity = rc.exception_handler(rc.compute_ROE, self.__financial_ratios, self.ttm, self.disp) 
        self.return_on_assets = rc.exception_handler(rc.compute_ROA, self.__financial_ratios, self.ttm, self.disp)
        self.return_on_capital_employed = rc.exception_handler(rc.compute_ROCE, self.__financial_ratios, self.ttm, self.disp)
        self.gross_margin = rc.exception_handler(rc.compute_gross_margin, self.__financial_ratios, self.ttm, self.disp)
        self.operating_margin = rc.exception_handler(rc.compute_operating_margin, self.__financial_ratios, self.ttm, self.disp)
        self.operating_cash_flow_ratio = rc.exception_handler(rc.compute_operating_cash_flow_ratio, self.__balance_sheet, self.__income_statement, self.ttm, self.disp)
        self.return_on_invested_capital = rc.exception_handler(rc.compute_return_on_invested_capital, self.__key_metrics, self.ttm, self.disp)
        self.return_on_sales = rc.exception_handler(rc.compute_return_on_sales, self.__financial_ratios, self.ttm, self.disp)
        
        #OPERATING ratios (also known as EFFICIENCY ratios)
        self.fixed_asset_turnover = rc.exception_handler(rc.compute_fixed_asset_turnover, self.__financial_ratios, self.ttm, self.disp)
        self.working_capital_turnover = rc.exception_handler(rc.compute_working_capital_turnover, self.__income_statement, self.__key_metrics, self.ttm, self.disp)
        self.total_asset_turnover = rc.exception_handler(rc.compute_total_asset_turnover, self.__financial_ratios, self.ttm, self.disp)
        self.inventory_turnover = rc.exception_handler(rc.compute_inventory_turnover, self.__financial_ratios, self.ttm, self.disp)
        self.days_of_inventory_on_hand = rc.exception_handler(rc.compute_days_of_inventory_on_hand, self.__key_metrics, self.ttm, self.disp)
        self.accounts_receivables_turnover = rc.exception_handler(rc.compute_accounts_receivables_turnover, self.__financial_ratios, self.ttm, self.disp)
        self.payables_turnover = rc.exception_handler(rc.compute_payables_turnover, self.__key_metrics, self.ttm, self.disp)
    
        #LEVERAGE ratios
        self.debt_to_assets = rc.exception_handler(rc.compute_debt_ratio, self.__financial_ratios, self.ttm, self.disp)
        self.interest_coverage = rc.exception_handler(rc.compute_interest_coverage, self.__key_metrics, self.ttm, self.disp)
        self.asset_to_equity = rc.exception_handler(rc.compute_asset_to_equity, self.__balance_sheet, self.ttm, self.disp)
        self.debt_to_equity = rc.exception_handler(rc.compute_debt_to_equity, self.__key_metrics, self.ttm, self.disp)
        self.equity_multiplier = rc.exception_handler(rc.compute_equity_multiplier, self.__financial_ratios, self.ttm, self.disp)
        self.total_debt_to_capitalization = rc.exception_handler(rc.compute_total_debt_to_capitalization, self.__financial_ratios, self.ttm, self.disp)
        self.total_debt_to_capital = rc.exception_handler(rc.compute_total_debt_to_capital, self.__balance_sheet, self.ttm, self.disp)
        self.net_debt_to_EBITDA = rc.exception_handler(rc.compute_net_debt_to_ebitda, self.__key_metrics, self.ttm, self.disp)
        self.degree_financial_leverage = rc.exception_handler(rc.compute_degree_financial_leverage, self.__growth, self.ttm, self.disp)
        
        #VALUATION ratios (also known as MARKET VALUE ratios)
        self.earnings_per_share = rc.exception_handler(rc.compute_earnings_per_share, self.__income_statement, self.ttm, self.disp)
        self.book_value_per_share = rc.exception_handler(rc.compute_book_value_per_share, self.__key_metrics, self.ttm, self.disp)
        self.cash_earnings_per_share = rc.exception_handler(rc.compute_cash_earnings_per_share, self.__key_metrics, self.ttm, self.disp)
        self.price_to_earnings = rc.exception_handler(rc.compute_price_to_earnings, self.__key_metrics, self.ttm, self.disp)
        self.price_to_book_value = rc.exception_handler(rc.compute_price_to_book_value, self.__financial_ratios, self.ttm, self.disp)
        self.price_to_sales = rc.exception_handler(rc.compute_price_to_sales, self.__financial_ratios, self.ttm, self.disp)
        self.price_to_free_cash_flow = rc.exception_handler(rc.compute_price_to_free_cash_flow, self.__financial_ratios, self.ttm, self.disp)
        self.price_earnings_to_growth = rc.exception_handler(rc.compute_price_earnings_to_growth, self.__financial_ratios, self.ttm, self.disp)
        self.dividend_yield = rc.exception_handler(rc.compute_dividend_yield, self.__financial_ratios, self.ttm, self.disp)
        self.dividend_payout = rc.exception_handler(rc.compute_dividend_payout, self.__financial_ratios, self.ttm, self.disp)
        self.enterprise_value_to_ebitda = rc.exception_handler(rc.compute_enterprise_value_to_ebitda, self.__key_metrics, self.ttm, self.disp)
        self.enterprise_value_to_sales = rc.exception_handler(rc.compute_enterprise_value_to_sales, self.__enterprise_value, self.__balance_sheet, self.__income_statement, self.ttm, self.disp)
        
        #LIQUIDITY ratios
        self.current_ratio = rc.exception_handler(rc.compute_current_ratio, self.__key_metrics, self.ttm, self.disp)
        self.quick_ratio = rc.exception_handler(rc.compute_quick_ratio, self.__financial_ratios, self.ttm, self.disp)
        self.cash_ratio = rc.exception_handler(rc.compute_cash_ratio, self.__financial_ratios, self.ttm, self.disp)
        self.days_of_sales_outstanding = rc.exception_handler(rc.compute_days_of_sales_outstanding, self.__key_metrics, self.ttm, self.disp)
        self.days_of_sales_in_inventory = rc.exception_handler(rc.compute_days_of_sales_in_inventory, self.__financial_ratios, self.ttm, self.disp)
        self.days_of_payables_outstanding = rc.exception_handler(rc.compute_days_of_payables_outstanding, self.__key_metrics, self.ttm, self.disp)
        self.operating_cycle = rc.exception_handler(rc.compute_operating_cycle, self.__financial_ratios, self.ttm, self.disp)
        self.cash_conversion_cycle = rc.exception_handler(rc.compute_cash_conversion_cycle, self.__financial_ratios, self.ttm, self.disp)

In [3]:
#Get S&P 500 companies

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

tickers_list = payload[0].Symbol.values.tolist()

#Replace 'BRK.B' by 'BRK-B' and 'BF.B' by 'BF-B'. The notation of the tickers can be checked with fa.available_companies(api_key)
tickers_list = list(map(lambda x: x.replace('BRK.B', 'BRK-B'), tickers_list))
tickers_list = list(map(lambda x: x.replace('BF.B', 'BF-B'), tickers_list))

In [4]:
tickers_list[:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD']

-------------

In [ ]:
count = 0
disp = 0
ttm = True
first_ticker = True
companies_dict = dict()

for ticker in tickers_list[count:]:
    
    company = Company(ticker, ttm, disp)
        
    companies_dict[ticker] = company.__dict__
    company_ratios_dict = dict(filter(lambda item: '__' not in item[0], vars(company).items()))

    if first_ticker: 
        df = pd.DataFrame(columns=list(company_ratios_dict.keys()))
        first_ticker = False
    df.loc[len(df)] = list(company_ratios_dict.values())
    
    print(f'Iteration: {count} ; Company: {ticker}')
    count+=1
    
    #Do some period sleeps to avoid overload the API
    if count%99 == 0:
        time.sleep(60)

In [50]:
df.head()

,ticker,ttm,disp,PAT_margin,EBITDA_margin,return_on_equity,return_on_assets,return_on_capital_employed,gross_margin,operating_margin,...,enterprise_value_to_ebitda,enterprise_value_to_sales,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_sales_in_inventory,days_of_payables_outstanding,operating_cycle,cash_conversion_cycle
0,MMM,True,9,0.147951,0.258702,0.255759,0.032777,0.058362,0.475317,0.216035,...,35.653272,1.956625,2.299583,1.4851,0.499276,52.036664,211.037581,43.89955,138.542168,94.642618
1,AOS,True,9,0.075956,0.141505,0.125759,0.016769,0.030725,0.344141,0.109227,...,39.399511,0.73837,2.295556,1.810996,0.586841,72.714417,294.897359,83.345285,120.567656,37.222371
2,ABT,True,9,0.143281,0.228433,0.205882,0.021755,0.038299,0.605988,0.188825,...,33.83372,1.32929,1.850375,1.352482,0.534156,78.006709,316.360542,39.192209,169.07422,129.882012
3,ABBV,True,9,0.259682,0.382428,0.785177,0.046318,0.076981,0.745298,0.325618,...,39.962747,None,2.235763,1.776078,0.951174,81.840976,331.910627,19.773545,161.113903,141.340359
4,ABMD,True,9,0.095609,0.124684,0.109801,0.022847,0.029078,0.800276,0.105313,...,151.912671,0.709353,5.449132,4.724765,0.312032,45.483376,184.460358,72.388229,207.954697,135.566468


**Because the process of loading the data took around 45 minutes running locally, it was usually splitted in two phases which created 2 sets of files that later were merged (see part 2)**

In [53]:
df.to_csv(f'(path file 1).csv')

In [54]:
#Save companies_dict in a pickle file
with open(f'(path file 1).pickle', 'wb') as file:
    pickle.dump(companies_dict, file)

<font color='cadetblue'><font size="4">**PART 2: CONCATENATE ALL CHUNCKS OF DATA TO BUILD A SINGLE DATASET**</font>

In [ ]:
#Define df_ratios from all csv files

files = ["(path file 1).csv", "(path file 2).csv"]
for file in files:
    load_file = pd.read_csv(filepath_or_buffer=file, index_col=0)
    if file == files[0]:
        df_ratios=load_file
        continue
    df_ratios = pd.concat([df_ratios, load_file], ignore_index=True, sort=False)

In [ ]:
df_ratios.to_csv('(path to save the final file).csv')

In [ ]:
#Define dict_companies from all pickle files

file_name_1 = '(path file 1).pickle'
file_name_2 = '(path file 2).pickle'

dict_companies = dict()
with open(file_name_1,'rb') as file:
    raw_data = file.read()
    dict_companies.update(pickle.loads(raw_data))
with open(file_name_2,'rb') as file:
    raw_data = file.read()
    dict_companies.update(pickle.loads(raw_data))

In [ ]:
len(dict_companies)

In [ ]:
with open(f'(path to save the final file).pickle', 'wb') as file:
    pickle.dump(dict_companies, file)